In [1]:
from rdflib import Namespace, Graph, URIRef, Literal
# Import pandas package

import pandas as pd
from difflib import SequenceMatcher

In [2]:
# change the cell width

pd.set_option("display.width", 120)

In [3]:
g_medextractor = Graph()
g_manual = Graph()

namespace_string = "http://fapranlp.de/"
namespace = Namespace(namespace_string)
# output_path = "../resources/manual_text_to_analyze.xml"
prefix = "fapra"
g_medextractor.parse("../MedExtractor/resources/evaluation_rdfs/depression.xml")
g_manual.parse("../MedExtractor/resources/manual_text_to_analyze.xml")
# print(g4)
for s, p, o in g_medextractor:
    print(s.replace(namespace_string,''),p.replace(namespace_string,''),o.replace(namespace_string+"symptom:",'').replace('_',' '))
print("----")
for s, p, o in g_manual:
    print(s.replace(namespace_string,''),p.replace(namespace_string,''),o.replace(namespace_string+"symptom:",'').replace('_',' '))
#     print((s, p, o))

depression hasSymptom lack of energy
depression hasSymptom interests
depression hasSymptom worried
depression hasSymptom low mood
depression hasSymptom feeling hopeless
depression hasSymptom no motivation
depression hasSymptom suicidal thoughts
depression hasSymptom interest
depression hasSymptom changes in appetite
depression hasSymptom aches
depression hasSymptom low sex drive
depression hasSymptom feeling guilt
depression hasSymptom constipation
depression hasSymptom pains
depression hasSymptom loss of libido
depression hasSymptom neglecting
depression hasSymptom psychological symptoms
depression hasSymptom low self esteem
depression hasSymptom tearful
depression hasSymptom feeling irritable
----
depression hasSymptom not getting any enjoyment out of life
depression hasSymptom low sex drive (loss of libido)
depression hasSymptom lack of energy
depression hasSymptom having no motivation or interest in things
depression hasSymptom disturbed sleep
depression hasSymptom changes to your 

In [4]:
data = {'Disorder': [],
    'Relationship': [],
    'Symptom': []}
  
  
# Convert the dictionary into DataFrame
df_manual = pd.DataFrame(data)
df_medextractor = pd.DataFrame(data)


for s, p, o in g_manual:
    d2 = {"Disorder": [s.replace(namespace_string,'')], 
          "Relationship": [p.replace(namespace_string,'')],
          "Symptom": [o.replace(namespace_string+"symptom:",'').replace('_',' ')]}
    df2 = pd.DataFrame(d2)
    df_manual = pd.concat([df_manual, df2],  ignore_index=True)
    
for s, p, o in g_medextractor:
    d3 = {"Disorder": [s.replace(namespace_string,'')], 
          "Relationship": [p.replace(namespace_string,'')],
          "Symptom": [o.replace(namespace_string+"symptom:",'').replace('_',' ')]}
    df3 = pd.DataFrame(d3)
    df_medextractor = pd.concat([df_medextractor, df3],  ignore_index=True)
print(df_manual)
print("----")
print(df_medextractor)

      Disorder Relationship                                            Symptom
0   depression   hasSymptom              not getting any enjoyment out of life
1   depression   hasSymptom                     low sex drive (loss of libido)
2   depression   hasSymptom                                     lack of energy
3   depression   hasSymptom         having no motivation or interest in things
4   depression   hasSymptom                                    disturbed sleep
5   depression   hasSymptom                    changes to your menstrual cycle
6   depression   hasSymptom                      avoiding contact with friends
7   depression   hasSymptom                                  suicidal thoughts
8   depression   hasSymptom  having difficulties in your home, work or fami...
9   depression   hasSymptom             taking part in fewer social activities
10  depression   hasSymptom          moving or speaking more slowly than usual
11  depression   hasSymptom             finding it d

In [5]:

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [6]:
df_manual['Similarity'] = 0
df_manual['Symptom_Medextractor'] = 0

for index, row in df_manual.iterrows(): 
    max_similarity = 0.0
    max_symptom = ""
    
    for x in df_medextractor['Symptom']:
        similarity = similar(row['Symptom'],x)
        if similarity > max_similarity:
            max_similarity = similarity
            max_symptom = x
    if max_similarity >= 0.40: 
        df_manual.loc[index, "Similarity"] = max_similarity
        df_manual.loc[index, "Symptom_Medextractor"] = max_symptom
        # else:
        #     df_manual.loc[index, "Similarity"] = ""
        #     df_manual.loc[index, "Symptom_Medextractor"] = ""
print(df_manual[['Disorder','Symptom','Symptom_Medextractor', 'Similarity']])

      Disorder                                            Symptom Symptom_Medextractor  Similarity
0   depression              not getting any enjoyment out of life                    0    0.000000
1   depression                     low sex drive (loss of libido)       loss of libido    0.636364
2   depression                                     lack of energy       lack of energy    1.000000
3   depression         having no motivation or interest in things        no motivation    0.472727
4   depression                                    disturbed sleep                    0    0.000000
5   depression                    changes to your menstrual cycle  changes in appetite    0.400000
6   depression                      avoiding contact with friends                    0    0.000000
7   depression                                  suicidal thoughts    suicidal thoughts    1.000000
8   depression  having difficulties in your home, work or fami...                    0    0.000000
9   depres

In [7]:
print(df_manual[['Disorder','Symptom','Symptom_Medextractor', 'Similarity']].to_latex(index=False))  

\begin{tabular}{lllr}
\toprule
  Disorder &                                            Symptom & Symptom\_Medextractor &  Similarity \\
\midrule
depression &              not getting any enjoyment out of life &                    0 &    0.000000 \\
depression &                     low sex drive (loss of libido) &       loss of libido &    0.636364 \\
depression &                                     lack of energy &       lack of energy &    1.000000 \\
depression &         having no motivation or interest in things &        no motivation &    0.472727 \\
depression &                                    disturbed sleep &                    0 &    0.000000 \\
depression &                    changes to your menstrual cycle &  changes in appetite &    0.400000 \\
depression &                      avoiding contact with friends &                    0 &    0.000000 \\
depression &                                  suicidal thoughts &    suicidal thoughts &    1.000000 \\
depression & having dif

true positive (TP) A test result that correctly indicates the presence of a condition or characteristic 
- medextractor found a symptom that is among the manually identified symptoms

true negative (TN) A test result that correctly indicates the absence of a condition or characteristic
- not relevant here

false positive (FP) A test result which wrongly indicates that a particular condition or attribute is present
- medextractor found a symptom that is not among the manually identified symptoms

false negative (FN) A test result which wrongly indicates that a particular condition or attribute is absent
- medextractor did not find a symptom that is among the manually identified symptoms

Precision = TP / (TP + FP)


Recall = TP / (TP + FN)


In [39]:
data = {'Symptom_Medextractor': [],
    'Original_Symptom': []}
df_result = pd.DataFrame(data)
original_symptom = "Original_Symptom"

for index, row in df_medextractor.iterrows(): 
    d_new = {"Symptom_Medextractor": [row['Symptom']], 
          original_symptom: ['']}
    dfnew = pd.DataFrame(d_new)
    df_result = pd.concat([df_result, dfnew],  ignore_index=True)
    
    for x in df_manual['Symptom']:
        if row['Symptom'] in x:
            if df_result.loc[index, original_symptom] == "":
                df_result.loc[index, original_symptom] = x
            else:
                df_result.loc[index, original_symptom] = df_result.loc[index, original_symptom] + ";" + x

print(df_result)

tp = (df_result[original_symptom] != '').sum()
fp = (df_result[original_symptom] == '').sum()
print("True positives: " + str(tp))
print("False positive: " + str(fp))
# Precision = TP / (TP + FP)
precision = tp / (tp + fp)
print("Precision: " + str(precision))

      Symptom_Medextractor                                   Original_Symptom
0           lack of energy                                     lack of energy
1                interests              neglecting your hobbies and interests
2                  worried                         feeling anxious or worried
3                 low mood                                continuous low mood
4         feeling hopeless                      feeling hopeless and helpless
5            no motivation         having no motivation or interest in things
6        suicidal thoughts                                  suicidal thoughts
7                 interest  having no motivation or interest in things;neg...
8      changes in appetite                      changes in appetite or weight
9                    aches                        unexplained aches and pains
10           low sex drive                     low sex drive (loss of libido)
11           feeling guilt                               feeling

In [40]:
print(df_result[['Symptom_Medextractor',original_symptom]].to_latex(index=False))  

\begin{tabular}{ll}
\toprule
  Symptom\_Medextractor &                                   Original\_Symptom \\
\midrule
        lack of energy &                                     lack of energy \\
             interests &              neglecting your hobbies and interests \\
               worried &                         feeling anxious or worried \\
              low mood &                                continuous low mood \\
      feeling hopeless &                      feeling hopeless and helpless \\
         no motivation &         having no motivation or interest in things \\
     suicidal thoughts &                                  suicidal thoughts \\
              interest & having no motivation or interest in things;negl... \\
   changes in appetite &                      changes in appetite or weight \\
                 aches &                        unexplained aches and pains \\
         low sex drive &                     low sex drive (loss of libido) \\
         fee

In [36]:
data = {'Symptom_manual': [],
    'Symptom_found': []}
df_result = pd.DataFrame(data)
symptom_found = "Symptom_found"

for index, row in df_manual.iterrows(): 
    d_new = {"Symptom_manual": [row['Symptom']], 
          symptom_found: ['']}
    dfnew = pd.DataFrame(d_new)
    df_result = pd.concat([df_result, dfnew],  ignore_index=True)
    
    for x in df_medextractor['Symptom']:
        if x in row['Symptom']:
            if df_result.loc[index, symptom_found] == "":
                df_result.loc[index, symptom_found] = x
            else:
                df_result.loc[index,symptom_found] = df_result.loc[index, symptom_found] + ";" + x

print(df_result)

fn = (df_result[symptom_found] == '').sum()
# tp = (df_result[symptom_found] != '').sum()
print("True positives: " + str(tp))
print("False negatives: " + str(fn))
# Recall = TP / (TP + FN)
recall = tp / (tp + fn)
print("Recall: " + str(recall))

                                       Symptom_manual                  Symptom_found
0               not getting any enjoyment out of life                               
1                      low sex drive (loss of libido)   low sex drive;loss of libido
2                                      lack of energy                 lack of energy
3          having no motivation or interest in things         no motivation;interest
4                                     disturbed sleep                               
5                     changes to your menstrual cycle                               
6                       avoiding contact with friends                               
7                                   suicidal thoughts              suicidal thoughts
8   having difficulties in your home, work or fami...                               
9              taking part in fewer social activities                               
10          moving or speaking more slowly than usual            

In [37]:
print(df_result[['Symptom_manual','Symptom_found']].to_latex(index=False))  

\begin{tabular}{ll}
\toprule
                                    Symptom\_manual &                 Symptom\_found \\
\midrule
             not getting any enjoyment out of life &                               \\
                    low sex drive (loss of libido) &  low sex drive;loss of libido \\
                                    lack of energy &                lack of energy \\
        having no motivation or interest in things &        no motivation;interest \\
                                   disturbed sleep &                               \\
                   changes to your menstrual cycle &                               \\
                     avoiding contact with friends &                               \\
                                 suicidal thoughts &             suicidal thoughts \\
having difficulties in your home, work or famil... &                               \\
            taking part in fewer social activities &                               \\
         movin

In [46]:
result_data = {'Text': [],
    'Precision': [],
    'Recall':[]}
df_result = pd.DataFrame(result_data)
d_new = {'Text': ['Depression'],
    'Precision': ['{0:.3f}'.format(precision)],
    'Recall':['{0:.3f}'.format(recall)]}
dfnew = pd.DataFrame(d_new)
df_result = pd.concat([df_result, dfnew],  ignore_index=True)
print(df_result.to_latex(index=False))  

\begin{tabular}{lll}
\toprule
      Text & Precision & Recall \\
\midrule
Depression &     0.900 &  0.581 \\
\bottomrule
\end{tabular}

